# The aim of this notebook: Preface

This notebook is adapted from a coding test completed as part of an application to [Algomo](https://www.algomo.com/) (I think they're doing something really cool so feel free to check them out). 

The aim of this test is to match questions across languages (i.e. teach a program to recognise *'Where is the atm'* as the same thing as *'Où est le guichet automatique'*.

This was done using Google's **Multilingual Sentence Encoder**, and some data scraped from a [german neobank](https://n26.com/en-eu). The neobank has a very straightforward html structure making it easy to scrape. 

## Imports

In [3]:
%load_ext autotime

time: 0 ns (started: 2021-02-13 16:54:40 -00:00)


In [4]:
# Web Scraping
import requests
from bs4 import BeautifulSoup
import re
from tqdm import tqdm

# FAQ_ID labelling
import pandas as pd 
import numpy as np
from sklearn.cluster import OPTICS
from sklearn.preprocessing import MaxAbsScaler

# Data Visualisation
import matplotlib.pyplot as plt
import altair
import dtale

starting_urls = ['https://support.n26.com/de-de',
                 'https://support.n26.com/de-at',
                 'https://support.n26.com/en-at',
                 'https://support.n26.com/en-de',
                 'https://support.n26.com/en-it',
                 'https://support.n26.com/it-it',
                 'https://support.n26.com/en-eu',
                 'https://support.n26.com/en-fr',
                 'https://support.n26.com/fr-fr',
                 'https://support.n26.com/en-es',
                 'https://support.n26.com/es-es',
                 'https://support.n26.com/en-us',
                 'https://support.n26.com/en-gb'
                ]

time: 0 ns (started: 2021-02-13 16:54:41 -00:00)


In [6]:
import tensorflow as tf
# tf.enable_eager_execution()

import tensorflow_hub as hub

time: 234 ms (started: 2021-02-13 16:58:28 -00:00)


# Problem statement

The task is to find some data from a neobank named N26, and to match questions across languages. It's in affect a web scraping and clustering problem, which affected how this was approached. 

# Scraping the FAQ data from the helpdesk of N26 across all markets (40%)


### Crawl (40%)


In [ ]:
# set up dataframe for use in crawling
df = pd.DataFrame(columns=['url', 'market', 'title', 'content'])


# crawl, extract and record relevant information
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
i = 0
errors = []

for url in tqdm(starting_urls):
    
    # get faq links from page (it's only the first 6 that we need)
    page = requests.get(url, headers=headers)
    faq_soup = BeautifulSoup(page.content, 'html.parser')
    market = url[-5:]
    faq_links = [link['href'] for link in faq_soup.find_all('a') if link['href'].startswith('/'+ market +'/')][:6]
    
    # get titles and title links links from faq links
    for link in faq_links:
        q_page = requests.get(url+link[6:], headers) 
        q_soup = BeautifulSoup(q_page.content, 'html.parser')
        q_links = [link['href'] for link in q_soup.find_all('a') 
                   if link['href'].startswith('/'+ market +'/')
                   and link.string.endswith('?')]
        titles = [link.string for link in q_soup.find_all('a')
                       if link.string is not None and link.string.endswith('?')]
        
        # get answers from question links and put everything into the dataframe set up earlier
        for q, q_link in enumerate(q_links):
            try:
                a_page = requests.get(url + q_link[6:], headers)
                a_source = BeautifulSoup(a_page.content, 'html.parser')
                a = a_source.find(id = 'main').get_text().split('?')[1:][0]
                df.at[i,'url'] = url + q_link[6:]
                df.at[i,'market'] = market
                df.at[i,'title'] = titles[q]
                df.at[i,'content'] = a
                i= i +1
            except AttributeError as error:
                errors.append([f'Error:{error} for url: {url + q_link[6:]}'])
                continue
  



In [19]:
print('Number of errors:',len(errors))
if len(errors) > 0:
    print('Error example:',errors[0])

NameError: name 'errors' is not defined

### Data Exploration

In [7]:
df = pd.read_csv('extracted_data_n26.csv', index_col=0)

time: 62 ms (started: 2021-02-13 16:58:43 -00:00)


In [8]:
df_no_duplicates = df.drop_duplicates(subset=['content'])

time: 0 ns (started: 2021-02-13 16:58:44 -00:00)


In [9]:
df.head()

,url,market,title,content
0,https://support.n26.com/de-de/konto-und-person...,de-de,Kann ich in meinem Land ein N26 Konto eröffnen?,Wir bieten unsere Konten in folgenden Ländern ...
1,https://support.n26.com/de-de/konto-und-person...,de-de,"Kann ich ein Konto eröffnen, wenn ich außerhal...","Wenn du in Polen, Schweden, Dänemark, Norwegen..."
2,https://support.n26.com/de-de/konto-und-person...,de-de,Wie eröffne ich mein N26 Konto?,Du kannst ein N26 Konto in der App (auf deinem...
3,https://support.n26.com/de-de/konto-und-person...,de-de,Warum funktioniert meine Video-Verifizierung n...,Einige Tipps für eine erfolgreiche Video-Verif...
4,https://support.n26.com/de-de/konto-und-person...,de-de,Wie kann ich mich ausweisen?,"Als Bank müssen wir wissen, wer du bist. Desha..."


time: 32 ms (started: 2021-02-13 16:58:44 -00:00)


In [10]:
print('df length:', len(df))
print('df length without duplicates:',len(df_no_duplicates))

df length: 1041
df length without duplicates: 536
time: 0 ns (started: 2021-02-13 16:58:47 -00:00)


In [11]:
dtale.show(df, ignore_duplicate=True).open_browser()

time: 6.67 s (started: 2021-02-13 16:58:48 -00:00)


# Finding closest matches (60%)

Finding questions that match from a group of questions is effectively a clustering problem. As it is a clustering problem, we'll need some way of turning the text into something that can be 'clustered'; in machine learning, clustering means finding the distances in vector space between vectors, and giving the same label to vectors within a given distance. 

The workflow will look like this:



1. <b> Get universal encoder from tensorflow </b> - The FAQs are in different languages. The Universal Sentence Encoder will help us to understand the context around words, even in different languages. 
2.<b> _Embed the question answers_ </b>- In order to cluster our answers, we'll use the encoder to represent our answers in vector form. There will be 512 numbers in each vector, corresponding to 512 distances that describe our answers.
3. <b>_Scale the embeddings_ </b>- Using sentences for text embedding often results in the embeddings becoming diluted: it's harder to tell the meaning of sentences apart from each other than it is for words, and this will reflect in the embedding. Scaling the embeddings will preserve the relationship between the embeddings mathematically, whilst making the differences more stark.
4. <b>_Cluster the embeddings_ </b>- scikit-learn has many clustering algorithms. OPTICS and DBScan are optimised for high dimensional data, and OPTICS more so according to the [docs](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.OPTICS.html).
5. <b>_Attach the cluster labels to the FAQs_</b> - We'll put all this work into a dataframe to be exported.

We'll also look at how good the culstering is when there are no repeated questions, as this should force cross language matching better

### Get the encoder from Tensorflow Hub

Step 1. In order to match the question answers to each other, the universal sentence encoder provided by tensorflow was used to create vector representations of each sentence on the data set (i.e. to create sentence embeddings). We can get the universal sentence encoder from the tensorflow hub.

In [13]:
# for the purposes of the interview we stongly encourage you to use the universal sentence encoder
# The following code will setup everything you need and setup the encoder for you

# For full disclaimer this code has been taken from
# Semantic Similarity with TF-Hub Universal Encoder at tf hub


module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print (f"module {module_url} loaded")
def embed(input):
    return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded
time: 16.8 s (started: 2021-02-13 17:03:16 -00:00)


The answers from the scraped data dataframe will provide the basis for the embedding. Lets embed our answers and have a look at what we get.

In [14]:
messages = df.sample(n=3).content
message_tensors = embed(messages)

for i, tensor in enumerate(message_tensors[:3]):
    print(f'Start of message: {messages.to_list()[i].split(".")[0]} \n')
    print(f'Embedding size: {len(tensor)}')
    print(f'First 3 embeddings: {tensor[:3]}')
    print('Standard deviation:', np.std(tensor))
    print('\n'*2)

Start of message: Your N26 You(new tab) and N26 Metal(new tab) membership includes a wide range of insurance benefits, covered by our partner Allianz Global Assistance Europe 

Embedding size: 512
First 3 embeddings: [-0.06015113 -0.04349793  0.00300509]
Standard deviation: 0.044181168



Start of message: Se stai già aspettando una carta, attendi che sia trascorso il tempo di consegna stimato 

Embedding size: 512
First 3 embeddings: [-0.05814423 -0.05300142 -0.01185939]
Standard deviation: 0.044128824



Start of message: Puedes ingresar dinero en tu cuenta de varias formas, en cuanto hayas verificado tu identidad:MoneyBeam: recibe dinero instantáneo de otros usuarios de N26 

Embedding size: 512
First 3 embeddings: [-0.05544928 -0.06063871  0.02223106]
Standard deviation: 0.044192936



time: 1.03 s (started: 2021-02-13 17:03:33 -00:00)


As expected, the distances from answer to answer are really, really small. The standard deviation is comparable to the embedding values, implying the answers within the vector space aren't very well defined. This will drastically affect the quality of the clusters extracted, but we should still be able to get some reasonable results (maybe).

The following function will implement steps 2 to 5.

In [15]:
def match_qas(qas_across_markets_df):
    """    
    This function assigns an faq_id to the input data, thus grouping question
    answer pairs across languages. A locale is the language of a market
    
    Parameters:
       qas_across_markets (list[
                         (market_1,url_1,title_1,content_1),
                         (market_2,url_2,title_2,content_2)])
    
    
    Returns:
       matched_data (list[
                         (faq_id_1,locale_1,market_1,title_1,content_1),
                         (faq_id_1,locale_2,market_2,title_2,content_2)])
    """
    
    # The data has structure (questions with the same language will be grouped together, due to the structure of the site).
    # This structure may interfere with our clusters, so we shuffle the data to fix that.
    shuffled_df = qas_across_markets_df.sample(frac=1, random_state=42)
    
    # Step 2
    embeddings = embed(shuffled_df.content)
    
    # Step 3
    # MaxAbsScaler is used for no particular reason other than it's slightly simpler than the alternatives,
    # and plays nicer with OPTICS.
    scalar = MaxAbsScaler()
    scaled_embeddings = scalar.fit_transform(embeddings)
    
    # Step 4
    # OPTICS is used to cluster the data for reasons mentioned earlier.
    clusters = OPTICS(n_jobs=-1, min_cluster_size=4, metric='cosine' ).fit(scaled_embeddings)
    
    # Step 4
    # The labels from the clusters are put into a dataframe along with the rest of the relevant data.
    locale_market = shuffled_df.market.str.split(pat='-', expand  = True)

    data_dict = {'FAQ_id':clusters.labels_+1 , 'locale':locale_market[0], 'market': locale_market[1],
             'title':shuffled_df.title.to_list(), 'content':shuffled_df.content.to_list()}
    
    matched_data = pd.DataFrame.from_dict(data_dict)
    return matched_data
    

final_results = match_qas(df)

# Add a version with the duplicates dropped. This should reduce the amount of clusters 
# and better match questions across languages as some redundancy is removed 

final_results_2 = match_qas(df_no_duplicates)

c:\users\bless\.virtualenvs\dl_projects-geciuh6f\lib\site-packages\sklearn\cluster\_optics.py:803: RuntimeWarning:

divide by zero encountered in true_divide



time: 3min 4s (started: 2021-02-13 17:03:34 -00:00)


In [16]:
print('The amount of FAQ IDs found for scraped data:',len(final_results.FAQ_id.unique()))
print('The amount of FAQ IDs found for scraped data without duplicates:',len(final_results_2.FAQ_id.unique()))

The amount of FAQ IDs found for scraped data: 87
The amount of FAQ IDs found for scraped data without duplicates: 11
time: 0 ns (started: 2021-02-13 17:06:39 -00:00)


In [17]:
# clusters 1 to 4 showed some question matching
final_results_2.loc[(final_results_2.market == 'it') & (final_results_2.FAQ_id!=0)]

,FAQ_id,locale,market,title,content
494,8,it,it,Quando arriverà la mia carta?,La consegna standard impiega fino a 15 giorni ...
387,5,en,it,How to file a complaint?,Looking for support or have a questionIf you d...
506,9,it,it,Quanto tempo richiedono i trasferimenti?,ℹ️ Valido per tutti i paesi tranne Regno Unito...
495,8,it,it,Come attivare la carta?,Se stai attendendo la consegna della tua carta...
543,9,it,it,Come guadagnare soldi invitando amici in N26?,Puoi iniziare a invitare amici tramite app opp...
498,8,it,it,Come bloccare la mia carta N26?,Puoi bloccare e sbloccare le tue carte in qual...
491,9,it,it,Come funziona N26 Business?,"ℹ️ Vale per Austria, Belgio, Danimarca, Estoni..."
464,9,it,it,Posso aprire un conto se vivo al di fuori dell...,"Se vivi in Polonia, Svezia, Danimarca, Norvegi..."
487,9,it,it,Come passare a Metal?,"ℹ️ Disponibile in Germania, Austria, Francia, ..."
485,9,it,it,Quali sono i vantaggi di N26 Metal?,"ℹ️Disponibile in Germania, Austria, Francia, S..."


time: 31 ms (started: 2021-02-13 17:06:39 -00:00)


In this notebook we attempted to match questions across languages by clustering them with a scikit-learn implementation of OPTICS, a density-based clustering algorithm tuned to high-dimensional data such as ours. We met VERY limited success, due to the curse of dimensionality and the narrowness of the topic at hand.

There should have been a cluster for every question from the 6 FAQ links scraped, but instead only 7 were found 

Some matching between French and English was made, probably because they're the two closest languages to each other. The clustering seemed mainly to focus on the language/words used and not the context of the words, resulting in the massive bucket of left-over content in the last bucket.

Some tuning of OPTICS clustering would improve this a lot, but after a few different parameters were tried and tested the results didn't get any better or more defined than they are now. Perhaps another approach could produced a more informative clustering.

## Saving the data frames

We'll save the scraped data that was used for machine learning and the results of machine learning.

In [ ]:
extracted_data_fname = 'extracted_data_n26.csv'
final_results_fname = 'n26_with_faq_.csv'

In [ ]:
df.to_csv(extracted_data_fname)
final_results_2.to_csv(final_results_fname)